#Table of Contents
* [1. PyTimeODE](#1.-PyTimeODE)
* [2. Installing](#2.-Installing)
* [3. Usage](#3.-Usage)
	* [3.1 Adams-Bashforth-Milne (ABM)](#3.1-Adams-Bashforth-Milne-%28ABM%29)
	* [3.2 Split-Operator Evolution](#3.2-Split-Operator-Evolution)
* [4. Development Instructions](#4.-Development-Instructions)


# 1. PyTimeODE

Dynamical evolution of complex systems.

This package provides an interface to a set of ODE solvers for solving dynamical time-evolution problems. The original application was quantum dynamics via Gross-Pitaevski equations (GPE) and superfluid density functional theory (TDDFT), but the code is quite general and should be able to be easily used for a variety of problems.

Currently the codes only use fixed time-step method (not adaptive).

# 2. Installing

Presently you must manually install the ``pytimeode`` package or include it with your project.  I currently recommend including it as a subrepository managed by [myrepos](http://myrepos.branchable.com).  For example, I typically use the following line in a top-level ``.mrconfig`` file (use the second checkout if you access bitbucket with ssh keys):

```
[_ext/pytimeode]
checkout = hg clone 'https://bitbucket.org/mforbes/pytimeode' 'pytimeode'
#checkout = hg clone 'ssh://hg@bitbucket.org/mforbes/pytimeode' 'pytimeode'
```

Then running ``mr checkout`` from the toplevel will pull and update the latest version of ``pytimeode`` and put it in ``_ext/pytimeode``.  I then create and add a symlink to this in the top level (and add ``_ext`` to my ``.hgignore`` file) so that I can use the ``pytimeode`` module directly:

```bash
ln -s _ext/pytimeode/pytimeode pytimeode
hg add pytimeode .mrconfig .hgignore
```

You can install it with `pip` using my package index as follows:

```
pip install --find-links https://bitbucket.org/mforbes/mypi/ --user pytimeode
```

At a later date, when the package is release, it will be able to be installed directly with ``pip install pytimeode``.

# 3. Usage

To use the solvers, you must define a class that imeplements one of the interfaces defined in ``pytimeode.interfaces``.  At a minimum, you must provide the methods required by ``IStateMinimal``.  This gives the evolvers the ability to manipulate your state, making copies, scaling the state, etc.  Addional require functionality by the ``IState`` interface can be obtained from these by inheriting from  ``StateMixin`` (though you might eventually like to provide custom implementations of the ``IState`` interface for performance.)  A variety of other mixins are provided for implementing states from a numpy arrays (``ArrayStateMixin``), a Mapping or Sequence of data (``StatesMixin``), a Mapping or Sequence of arrays (``ArraysStateMixin``), or a Mapping or Sequence of other states (``MultiStateMixin``).

In [1]:
from nbutils import describe_interface
import pytimeode.interfaces
describe_interface(pytimeode.interfaces.IStateMinimal)

Then you must satisfy the requirements of your particular solver.  Currently we support the following solvers.

## 3.1 Adams-Bashforth-Milne (ABM)

In [2]:
describe_interface(pytimeode.interfaces.IStateForABMEvolvers)

## 3.2 Split-Operator Evolution

In [3]:
describe_interface(pytimeode.interfaces.IStateForSplitEvolvers)

# 4. Development Instructions

If you are a developer of this package, there are a few things to be aware of.

1. If you modify the notebooks in ``docs/notebooks`` then you may need to regenerate some of the ``.rst`` files and commit them so they appear on bitbucket.  This is done automatically by the ``pre-commit`` hook in ``.hgrc`` if you include this in your ``.hg/hgrc`` file with a line like:

    ```
    %include ../.hgrc
    ```

**Security Warning:** if you do this, be sure to inspect the ``.hgrc`` file carefully to make sure that no one inserts malicious code.